# Final Project - Python INSY 5336
### Prajwal Prasad
### 1001750483

<i>This block is mainly for the import of libraries necessary for this project and initializing the 3 website movie details lists.</i>

In [18]:
try:
    import urllib.request, urllib.parse, urllib.error #necessary imports for this project.
    from bs4 import BeautifulSoup
    import ssl
    import requests
    import csv
    import re

    # Ignore SSL certificate errors
    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

    metacritics_list = [] #Initialize 3 empty lists to store the movie details
    imdb_list=[]
    rt_list=[]
    metacritics_review_txt='' #Initialize 3 empty strings to store all the movie reviews
    imdb_review_txt=''
    rt_review_txt=''
    
except: #catch exceptions incase any occur, such as missing libraries.
    print('Failed to import one or more libraries, please check') 

<i>This block is mainly focused on MovieReviewDatabase database creation and connection using the sqlite connect function, and the MovieReviewTable creation using sqlite3 cursor's execute. </i>

In [19]:
import sqlite3 as lite

try:
    con = lite.connect('MovieReviewDatabase.db') #try to create and connect to the db
except:
    print('Unable to connect to the database!')

with con:
    try:
        cur = con.cursor()
        cur.execute("DROP TABLE IF EXISTS MovieReviewTable") #uncomment this incase table already exists and you want to re-test.
        cur.execute("CREATE TABLE MovieReviewTable(website TEXT, rank INT, movie TEXT, genre TEXT)") #creates a table with the given attributes and attribute type.
        print("Table created successfully!")
    except:
        print('Table already exists, please execute drop table first or you can skip this block!')

Table created successfully!


<i>The fetchMetaCritics(), fetchIMDB(), fetchRT() all work very similarly and are used to fetch the top 50 movies from each of the 3 websites. First, we find out the url for the top movies of each of the website, using inspect element on chrome we can get the html tags where the information we seek are embedded. requests is used to create a HTTP request using the supplied headers, the output of that is parsed by beautifulsoup. Once we get the top 100/250 or whatever movies on the website, we narrow it down to 50 results. For each movie in that result we find the movie title using the find(), and the url of that movie which is usually in href of the movie title. 

With the movie url we call the getMovieDetails(). In there, we can create another request to get the details of that movie such as genre, reviews etc. As we had done before, we find the tags of the content we need (in this case genre and reviews) and parse them using beatifulsoup. The text we get is sometimes unformatted with lots of spaces and special characters that can mess up our analysis, using regular expressions we can substitute any special characters with blanks. There can be multiple reviews on a single movie so we hit 50 of them and create a single large text block of reviews which is added to the website review string and then return the genre back to the calling function.

With all the fields acquired for the movie, we can append it to the main movie details list of that website. Also, create a new csv file to store the movie details and write the entire list using writerows function in one shot. I have commented the csv writing part since I am already providing the csv with loaded data, you can uncomment and try it out as well! </i>


In [20]:
#This code block is for scraping from metacritics website.
def fetchMetaCritics():

    def get_MC_MovieDetails(link): #for fetching genre and reviews of each induvidual movie
        page = requests.get(link, headers = headers)
        soup = BeautifulSoup(page.content, "html.parser")
        genre_lst = soup.find('div', class_ = 'genres').find_all('span') #for genre
        if(len(genre_lst)<1):
            raise Exception('Invalid tag name for genre')
        genre_lst=[str(i) for i in genre_lst[2:]] #convert each tag to string object
        genre= ','.join(genre_lst) #convert list to a single string
        genre=genre.replace('<span>','')
        genre=genre.replace('</span>','') #remove all the tags
        
        try:
            r_link = 'https://www.metacritic.com' + soup.find_all('a', class_ = 'see_all boxed oswald')[1].get('href', None) #the users review link
            r_page = requests.get(r_link, headers = headers)
            r_soup = BeautifulSoup(r_page.content, "html.parser")
            review_lst = r_soup.find('div', class_ = 'user_reviews').find_all('div',class_='review_body') # for reviews fetch from user reviews.
        except:
            r_link = 'https://www.metacritic.com' + soup.find_all('a', class_ = 'see_all boxed oswald')[0].get('href', None) #the critics review link incase there are no user reviews
            r_page = requests.get(r_link, headers = headers)
            r_soup = BeautifulSoup(r_page.content, "html.parser")
            review_lst = r_soup.find('div', class_ = 'critic_reviews').find_all('div',class_='summary') # for reviews fetch from critics reviews.
        
        if(len(review_lst)<1):
            raise Exception('Invalid tag name for review')
            
        for rv in review_lst[:50]:#Get 50 reviews from the review list
            text=rv.text
            text=text.replace('Expand','')
            text=text.replace('Read full review','')
            text=text.replace('\n','')
            text=text.strip() #remove blanks
            text = re.sub('[^a-zA-Z0-9 \n\.]', '', text) #remove special characters
            global metacritics_review_txt
            metacritics_review_txt+= ' '+text.lower()#Keep adding each review to the large review_txt, lower case so its easier to generate frequency of words.
        
        return genre  #return only genre back to the calling function 
        
            

    try:
        metacritics_url='https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc' #url for top 100 movies.
        page = requests.get(metacritics_url, headers = headers) #headers so we make a legit query and not as a bot
        soup = BeautifulSoup(page.content, "html.parser")
        container = soup.find_all('td', class_ = 'clamp-summary-wrap')
        if(len(container)<1):
            raise Exception('Unable to fetch movies.Please make sure the URL is correct')
        movies = container[:50] #here we get the top 50 movies we want
        rank=1
        for movie in movies: #for each movie in the top 50 store it into the metacritics list with website,rank,movie,genre
            temp=[]
            name = movie.find('h3').text #movie title
            if(len(name)<1):
                raise Exception('Tag name invalid ',movie)
            temp.append("metacritics")
            temp.append(rank)
            temp.append(name)
            rank+=1
            tag = movie.find('a', class_ = 'title')
            link = tag.get('href', None)
            if(len(link)<1):
                raise Exception('link not found ',movie)
            temp.append(get_MC_MovieDetails('https://www.metacritic.com'+link)) #fetch the movie from the href link in movie title store genre in temp
            metacritics_list.append(temp) #append temp which contains each movie genre info to the main metacritics list
            
            
        print("Writing Metacritic reviews to CSV") #create a new csv file and write all the metacritics movies and reviews.
        with open('prajwal_reviews.csv', 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(metacritics_list)
        print("Done.")
        
    except Exception as e:
        print(e)

fetchMetaCritics() #invoke the main function

Writing Metacritic reviews to CSV
Done.


In [21]:
#This code block is for scraping from IMDB website.

def fetchIMDB():
    
    def get_imdb_MovieDetails(link): #for fetching genre and reviews of each induvidual movie
        
        page = requests.get(link, headers = headers)
        soup = BeautifulSoup(page.content, "html.parser")
        genre_lst = soup.find('div', class_ = 'subtext').find_all('a') #for genre
        if(len(genre_lst)<1):
            raise Exception('Invalid tag name for genre')
        genre=''
        del genre_lst[-1] #there was another anchor tag which didnt have genre info
        for gl in genre_lst:
            genre+= gl.text +"," #generate string from list, I wanted to change the way I did from previous block
        genre=genre[:-1]
        r_link= 'https://www.imdb.com' + soup.find('div', class_ = 'user-comments').find_all('a')[-1].get('href', None)
        r_page = requests.get(r_link, headers = headers)
        r_soup = BeautifulSoup(r_page.content, "html.parser")     
        review_lst =r_soup.find('div', class_ = 'lister-list').find_all('div',class_='text show-more__control') # for reviews fetch from user reviews.
        tag = r_soup.find('div',class_ = 'load-more-data')
        ajaxurl = tag.get('data-ajaxurl',None)
        datakey = tag.get('data-key',None)
        second_page= f'https://www.imdb.com/{ajaxurl}?paginationKey={datakey}' #Repeat the same process to get the content from 2nd page using ajax
        r_page = requests.get(second_page, headers = headers)
        r_soup = BeautifulSoup(r_page.content, "html.parser")    
        review_lst.extend(r_soup.find('div', class_ = 'lister-list').find_all('div',class_='text show-more__control')) # for 25 more reviews fetch from user reviews.

        if(len(review_lst)<1):
            raise Exception('Invalid tag name for review')
        
        for rv in review_lst[:50]:  #Get 50 reviews from the review list
            text=rv.text
            text=text.replace('Expand','')
            text=text.strip() #remove blanks
            text = re.sub('[^a-zA-Z0-9 \n\.]', '', text) #remove special characters
            global imdb_review_txt
            imdb_review_txt+= ' '+text.lower() #Keep adding each review to the large review_txt, lower case so its easier to generate frequency of words.
            
        return genre #return only genre back to the calling function

    
    
    try:
        imdb_url='https://www.imdb.com/chart/top/?ref_=nv_mv_250'#url for top 250 movies.
        page = requests.get(imdb_url, headers = headers) #headers so we make a legit query and not as a bot
        soup = BeautifulSoup(page.content, "html.parser")
        container = soup.find_all('td', class_ = 'titleColumn')
        if(len(container)<1):
                raise Exception('Unable to fetch movies.Please make sure the URL is correct')
        movies = container[:50] #here we get the top 50 movies we want
        rank=1
        for movie in movies: #for each movie in the top 50 store it into the imdb list with website,rank,movie,genre
            temp=[]
            name = movie.find('a').text #movie title
            if(len(name)<1):
                raise Exception('Tag name invalid ',movie)
            temp.append("imdb")
            temp.append(rank)
            temp.append(name)
            rank+=1
            tag = movie.find('a')
            link = tag.get('href', None)
            if(len(link)<1):
                raise Exception('link not found for ',movie)
            temp.append(get_imdb_MovieDetails('https://www.imdb.com'+link)) #fetch the movie from the href link in movie title store genre in temp
            imdb_list.append(temp) #append temp which contains each movie genre info to the main imdb list
            

        print("Writing IMDB reviews to CSV")
        with open('prajwal_reviews.csv', 'a', newline='') as file: #Open csv file and append imdb movies without overwriting all previous data.
            writer = csv.writer(file)
            writer.writerows(imdb_list)
        print("Done.")  
        
    except Exception as e:
        print(e)
    
    
fetchIMDB() #invoke the main function


Writing IMDB reviews to CSV
Done.


In [22]:
#This code block is for scraping from Rotten Tomatoes website.
def fetchRT():
    
    def get_RT_MovieDetails(link):#for fetching genre and reviews of each induvidual movie
        page = requests.get(link, headers = headers)
        soup = BeautifulSoup(page.content, "html.parser")
        genre_lst = soup.find('div', class_ = 'meta-value genre').text #for genre
        if(len(genre_lst)<1):
            raise Exception('Invalid tag name for genre')
        genre_lst= genre_lst.split(',')
        genre=''
        for g in genre_lst: #put all genres into comma sepeated string
            g=g.strip()
            g=g.capitalize() #the genre text was in lower case, changed to match with other websites.
            genre+=g +','
        genre=genre[:-1]
        review_lst=[]
        
        for i in range(1,4): #Fetch reviews from pages 1-4 (This is enough to get 50 reviews)
            try:
                r_link = link + '/reviews?&page=' + str(i)
                r_page = requests.get(r_link, headers = headers)
                r_soup = BeautifulSoup(r_page.content, "html.parser")
                review_lst.extend(r_soup.find('div', class_ = 'review_table').find_all('div',class_='the_review')) # for reviews fetch from all critic reviews.
            except:
                continue
                
        if(len(review_lst)<1):
            raise Exception('Invalid tag name for review')
        
        for review in review_lst[:50]:#Get 50 reviews from the review list
            text = review.text
            text = text.strip()
            text = re.sub('[^a-zA-Z0-9 \n\.]', '', text) #remove blanks and special chars
            global rt_review_txt
            rt_review_txt += ' '+text.lower()  #Keep adding each review to the large review_txt, lower case so its easier to generate frequency of words.
        

        return genre #return only genre back to the calling function
        
    
    try:
        rt_url='https://www.rottentomatoes.com/top/bestofrt/' #url for top 100 movies.
        page = requests.get(rt_url, headers = headers) #headers so we make a legit query and not as a bot
        soup = BeautifulSoup(page.content, "html.parser")
        container = soup.find('table',class_='table').find_all('a', class_ = 'unstyled articleLink')
        if(len(container)<1):
                raise Exception('Unable to fetch movies.Please make sure the URL is correct')
        movies= container[:50]
        rank=1
        for movie in movies: #for each movie in the top 50 store it into the rt list with website,rank,movie,genre,reviews
            temp=[]
            temp.append("rotten-tomatoes")
            temp.append(rank)
            name=movie.text
            if(len(name)<1):
                raise Exception('Tag name invalid ',movie)
            name=name[:-6].strip()
            temp.append(name)
            rank+=1
            link = movie.get('href', None)
            if(len(link)<1):
                raise Exception('link not found for ',movie)
            temp.append(get_RT_MovieDetails('https://www.rottentomatoes.com'+link)) #fetch the movie from the href link in movie title store genre in temp
            rt_list.append(temp)  #append temp which contains each movie genre info to the main rt list

        print("Writing Rotten Tomatoes reviews to CSV") #Open csv file and append imdb movies without overwriting all previous data.
        with open('prajwal_reviews.csv', 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(rt_list)
        print("Done.")
        
    except Exception as e:
        print(e)

fetchRT() #invoke the main function

Writing Rotten Tomatoes reviews to CSV
Done.


<i>This block is focused on writing all the data we acquired thus far into the MovieReviewTable. I have used sql parameterized insert query and cursor's executemany function, so its a lot easier to push all the movie data of each website in just 3 insert statements! </i>

In [23]:
#This block is for writing all movies data into the database. Don't run it if you did not drop the table.
try:
    with con:
        mc_tuple = tuple(tuple(sub) for sub in metacritics_list) #convert list of list into tuple of tuples for prepared statement, makes it lot easier to insert.
        imdb_tuple = tuple(tuple(sub) for sub in imdb_list)
        rt_tuple = tuple(tuple(sub) for sub in rt_list)
        cur = con.cursor()
        print("writing into table")
        cur.executemany("INSERT INTO MovieReviewTable VALUES(?, ?, ?, ?)", mc_tuple) #inserting each of the movie details from 3 websites into the table.
        cur.executemany("INSERT INTO MovieReviewTable VALUES(?, ?, ?, ?)", imdb_tuple)
        cur.executemany("INSERT INTO MovieReviewTable VALUES(?, ?, ?, ?)", rt_tuple)
        con.commit()
        print("write successful")
        
except Exception as e: #catch any exceptions that occur
    if type(e).__name__== 'NameError': #incase previous code blocks were not executed.
        print('Unable to establish connection to Database, Please execute code blocks in order.')
    else:
        print(e)


writing into table
write successful


<i>We create a frequency of words dictionary using the movie review text we had generated for each website. We take the reviews of each movie and after some text processing split the review text (containg all movie reviews) into words. The words are stored and counted in a dictionary, which is sorted on desencding order to get the top 50 words which occur in the text.</i>

In [24]:
#Question 1

def getTopWords(big_text): #create a dictionary counter of words using the movie reviews of each website.

    if(len(big_text)<1):
        raise Exception('Empty String.')

    #stop words which add no value, and repeated often.
    stop_words = set (["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", '', ' ', "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"])

    big_text=big_text.replace('.',' ') #replace periods with blanks in case words appear conjoint
    words=big_text.split(' ')

    dict={}

    for word in words: #generate frequency of words which are not stop words
        if word in dict:
            dict[word]=dict[word]+1
        elif word not in stop_words:
            dict[word]=1

    if(len(dict)<1):
        raise Exception('The dictionary turned up empty, check if the lists are generated properly.')

    top_50=sorted(dict.items(), key=lambda x: x[1], reverse=True) #get top 50 in decending order.
    top_50=top_50[:50]

    ret_dict={}
    for item in top_50: #dictionary of word as key and fequency of occurance as value.
        ret_dict[item[0]]=item[1]

    return ret_dict


try: #generate top 50 used words for each website.
    imdb_wordDict = getTopWords(imdb_review_txt) #each review_txt contains all the reviews for top 50 movies of that website
    metacritics_wordDict = getTopWords(metacritics_review_txt)
    rt_wordDict = getTopWords(rt_review_txt)

    print("rotten tomatoes",rt_wordDict)
    print("imdb",imdb_wordDict)
    print("metacritics",metacritics_wordDict)
    
except Exception as e:
    if type(e).__name__== 'NameError':
        print('Please make sure to run the code blocks in order.')
    else:
        print(e)

rotten tomatoes {'film': 504, 'one': 312, 'movie': 242, 'full': 192, 'story': 190, 'review': 173, 'spanish': 167, 'best': 161, 'films': 159, 'like': 154, 'great': 122, 'time': 114, 'ever': 100, 'good': 95, 'still': 87, 'movies': 84, 'comedy': 78, 'even': 75, 'way': 75, 'never': 75, 'much': 74, 'life': 72, 'classic': 72, 'us': 71, 'action': 69, 'see': 69, 'made': 69, 'new': 68, 'also': 68, 'well': 66, 'funny': 66, 'many': 65, 'fun': 65, 'first': 64, 'love': 64, 'characters': 63, 'performances': 63, 'family': 63, 'director': 62, 'world': 61, 'seen': 59, 'every': 58, 'superhero': 58, 'makes': 58, 'get': 56, 'work': 56, 'something': 55, 'horror': 55, 'cinema': 54, 'emotional': 53}
imdb {'film': 6135, 'movie': 5466, 'one': 3868, 'time': 2087, 'like': 1953, 'story': 1828, 'best': 1746, 'great': 1624, 'see': 1505, 'even': 1495, 'good': 1419, 'films': 1374, 'first': 1363, 'would': 1349, 'really': 1261, 'also': 1260, 'well': 1246, 'movies': 1215, 'much': 1213, 'life': 1205, 'ever': 1196, 'chara

<i>Once we get the top 50 words and their frequency  of each website, we can find out the similarity between them using the cosine similarity formula. I am using numpy to ease the calculations required. First sets of top 50 words for both inputs are created, followed by a set union to combine them and remove duplicates. For each word in the union set, we add the frequency of that word in each dictionary to the respective list (l1 for dict1 and l2 for dict2) or 0 if it doesn't occur in that dictionary. The formula is then applied on the 2 lists to get a value between 0 and 1.</i>

In [25]:
#calculate cosine similarity of reviews between 2 movie websites using their dictionaries as input
def similarityCalc(dict1,dict2): 
    from numpy import dot #numpy functions for cosine similarity formula.
    from numpy.linalg import norm

    l1,l2=[],[]

    set1 = set(list(dict1.keys()))#convert list of keys to set so we can perform set operations needed for cos_sim.
    set2 = set(list(dict2.keys()))

    if(len(set1)<1 or len(set2)<1):
        raise Exception('Set can not be empty.')

    rvector = list(set1.union(set2))  #combine unique keys in both websites.

    for w in rvector: #generate 2 lists with frequency values of each words in both websites respectively.
        if w in set1: 
            l1.append(dict1[w]) #get the freq of word in dict 1
        else: 
            l1.append(0) 
        if w in set2: 
            l2.append(dict2[w]) #get the freq of word in dict 2
        else: 
            l2.append(0) 
    c = 0

    cos_sim = dot(l1, l2)/(norm(l1)*norm(l2)) # the cosing similarity formula
    if(cos_sim > 1 or cos_sim < 0):
        raise Exception('Something went wrong.cos_similarty value can only be between 0 and 1')
    return cos_sim


try:
    print("Similarity score for keywords between imdb and metacritics is: ",similarityCalc(imdb_wordDict,metacritics_wordDict))
    print("Similarity score for keywords between imdb and rotten tomatoes is: ",similarityCalc(imdb_wordDict,rt_wordDict))
    print("Similarity score for keywords between rotten tomatoes and metacritics is: ",similarityCalc(rt_wordDict,metacritics_wordDict))

except Exception as e:
    if type(e).__name__== 'NameError':
        print('Please make sure to run the code blocks in order.')
    else:
        print(e)
    
    
    

Similarity score for keywords between imdb and metacritics is:  0.953053201178429
Similarity score for keywords between imdb and rotten tomatoes is:  0.8274495152606323
Similarity score for keywords between rotten tomatoes and metacritics is:  0.803066869486583


<i> We the do same process of finding similarity but with genres this time. Using the movie details list of each website, we create the genre dictionary using the generateGenreCounter(). For each movie in the top 50, we split the genres and add to a dictionary with acts as a counter. After we get the frequency of each genre in each website, we apply the same similarityCalc() to get the cosine similiarty of genres between two websites. </i>

In [26]:
#Question 2.
      
def generateGenreCounter(lst): #create a frequency counter of genres in each website
    if(len(lst)<1):
        raise Exception('List can not be empty.')

    ret={}

    for item in lst:
        if(len(item)<3):
            raise Exception('Make sure to use the correct list with all the movie details!')
        genres = item[3] #index 3 has the movie's genre info
        genres = genres.split(',')
        for genre in genres: #incase the movie has more than 1 genres split them and add seperately.
            if genre in ret:
                ret[genre] = ret[genre] + 1 
            else:
                ret[genre] = 1
    return ret
    

       
try:
    imdb_genre_dict =generateGenreCounter(imdb_list) #genereate each movies genre dict using their movie details list as input
    metacritics_genre_dict =generateGenreCounter(metacritics_list)
    rt_genre_dict =generateGenreCounter(rt_list)

    print("Similarity score for genres between imdb and metacritics is: ",similarityCalc(imdb_genre_dict,metacritics_genre_dict))
    print("Similarity score for genres between imdb and rotten tomatoes is: ",similarityCalc(imdb_genre_dict,rt_genre_dict))
    print("Similarity score for genres between rotten tomatoes and metacritics is: ",similarityCalc(rt_genre_dict,metacritics_genre_dict))

except Exception as e:
    if type(e).__name__== 'NameError':
        print('Please make sure to run the code blocks in order.')
    else:
        print(e)

Similarity score for genres between imdb and metacritics is:  0.8857105210415298
Similarity score for genres between imdb and rotten tomatoes is:  0.7707048953466868
Similarity score for genres between rotten tomatoes and metacritics is:  0.6824329998331107
